# Blog with images

We use this example of creating a web search-based blog with illustrations to show how to 

* give agents as tools to other agents
* create tasks and link them using the >> operator

Let's install external dependencies for the example.

In [1]:
%pip install duckduckgo-search

# install lunary if you need logging
# %pip install lunary==1.1.5


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path
import os
import sys

from dotenv import load_dotenv
from langchain_community.tools import DuckDuckGoSearchRun

WORKING_DIR = Path(os.path.realpath("."))

try:
    from motleycrew import MotleyCrew
except ImportError:
    # if we are running this from source
    motleycrew_location = os.path.realpath(WORKING_DIR / "..")
    sys.path.append(motleycrew_location)

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.tools import MotleyTool, RetryConfig
from motleycrew.tools.image.dall_e import DallEImageGeneratorTool
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

configure_logging()
load_dotenv()

/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/lunary/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


True

First, we create a fairly conventional agent that has a search tool. 

In [3]:
search_tool = MotleyTool.from_supported_tool(
    DuckDuckGoSearchRun(), retry_config=RetryConfig(max_retries=7)  # for retrying rate limit errors
)

# The name and description are helpers, they are not included in the prompt
researcher = ReActToolCallingMotleyAgent(
    name="Senior Research Analyst",
    description="Agent that conducts research on AI and data science",
    prompt="You work at a leading tech think tank. "
    "Your expertise lies in identifying emerging trends. "
    "You have a knack for dissecting complex data and presenting actionable insights. "
    "You uncover cutting-edge developments in AI and data science, doing web search if necessary. "
    "\n\n{prompt}",
    verbose=True,
    tools=[search_tool],
)

Now the fun starts happening: we pass that agent to another agent as a tool. That's how delegation works in motleycrew: you just pass agents as tools to other agents; the API is automatically converted under the hood.

In [4]:
# You can give agents as tools to other agents
writer = ReActToolCallingMotleyAgent(
    name="AI writer agent",
    prompt="You are an expert writer for a tech blog. "
    "You like to write about complex topics in a way that is easy to understand, "
    "with the help of the researcher agent."
    "\n\n{prompt}",
    tools=[researcher],
    verbose=True,
)


The `{prompt}` placeholders are where the task-specific prompts will go.

Let's create a first task, writing a blog post, and give it to the writer agent. We'll need to instantiate a crew first as a crew orchestrates the tasks.

In [5]:
# You always need a crew to orchestrate the agents
crew = MotleyCrew()

blog_post_task = SimpleTask(
    crew=crew,
    name="produce blog post on AI advancements",
    description="Develop an engaging blog post that highlights the latest AI advancements in 2025. "
    "Your post should be informative yet accessible, catering to a tech-savvy audience. "
    "Make it sound cool, avoid complex words so it doesn't sound like AI. "
    "Create a blog post of at least 4 paragraphs, in markdown format.",
    agent=writer,
)

Now let's create a second task, to generate and insert into the blog some illustrations.

In [6]:
# Illustrator
image_generator_tool = DallEImageGeneratorTool(os.path.realpath("./images"),
                                               model="dall-e-3",
                                               refine_prompt_with_llm=True,
                                               size="1024x1024",
                                               style="vivid")

illustrator = ReActToolCallingMotleyAgent(
    name="Illustrator",
    description="Create beautiful and insightful illustrations for a blog post",
    tools=[image_generator_tool],
)

illustration_task = SimpleTask(
    crew=crew,
    name="create an illustration for the blog post",
    description="Create beautiful and insightful illustrations to accompany the blog post on AI advancements. "
    "The blog post will be provided to you in markdown format. "
    "Make sure to use the illustration tool provided to you, once per illustration, "
    "and embed the URL provided by the tool into the blog post. "
    "Create between 2 and 3 illustrations, neither more nor less.\n"
    "Only return the full text of the blog post with embedded URLs, no backticks, no other text. "
    "Only use slashes as path separators, never backslashes",
    agent=illustrator,
)

Now let's connect the two tasks, and run them!

In [ ]:
# Make sure the illustration task runs only once the blog post task is complete, and gets its input
blog_post_task >> illustration_task

# Get your crew to work!
result = crew.run()



> Entering new AgentExecutor chain...

Invoking: `senior_research_analyst` with `{'input': 'latest AI advancements in 2025'}`
responded: Thought: To write an engaging blog post about the latest AI advancements in 2025, I need to gather information on the most recent developments and breakthroughs in AI technology. This will help ensure that the content is both current and relevant. I will use the senior_research_analyst tool to research the latest AI advancements in 2025.

Let's proceed with gathering this information.



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'latest AI advancements 2025'}`
responded: Thought: To provide the latest AI advancements in 2025, I need to perform a web search to gather current information on this topic. I will use the DuckDuckGo search tool to find relevant and up-to-date information about AI developments in 2025.





/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Jul 24, 2025 · PwC’s mid-year AI check-in: get the latest on 2025 AI predictions, discover which trends are accelerating or fading, and gain action-ready insights to stay ahead. May 21, 2025 · Many of the most important AI trends in the first half of 2025 reflect changes in how the industry is applying existing ideas—some pragmatic and productive, others less so. … Jan 3, 2025 · Explore the key trends shaping AI in 2025, from multimodal models and AI agents to security challenges and evolving regulatory landscapes. Jun 23, 2025 · In 2025, there’s a new AI tool for everything – text, images, coding, video, you name it, and professionals are eager to know “what’s the best tool for making their work easy?” This … Jun 13, 2025 · The breakthroughs span across healthcare, finance, transportation, robotics, communication, and more. As leading industry pioneers, we explore the most impactful AI …Final Answer: In 2025, AI advancements are characterized by several key trends and developments:

1

Let's display the result (swapping absolute for relative paths, as Jupyter seems to struggle with the former)

In [8]:
from IPython.display import Markdown, display
here = os.path.realpath(".").replace(os.sep,"/")
display(Markdown(illustration_task.output.replace(here,".")))

Final Answer: 

# The Coolest AI Advancements of 2025: A Peek into the Future

Welcome to 2025, where AI is not just a buzzword but a transformative force reshaping our world. This year, AI has taken some giant leaps, making it an exciting time for tech enthusiasts and professionals alike. Let's dive into the coolest advancements that are making waves in the AI landscape.

## Multimodal Models and AI Agents

Imagine an AI that can understand and process text, images, and audio all at once. That's the power of multimodal models, a major trend in 2025. These AI systems are like supercharged assistants, capable of performing complex tasks by integrating different types of data. Whether it's helping doctors diagnose diseases by analyzing medical images and patient records or creating immersive virtual experiences, these AI agents are setting new standards for what machines can do.

![Multimodal Models and AI Agents](./images/eea26ec6.png)

## Tailored AI for Every Industry

AI is no longer a one-size-fits-all solution. In 2025, we're seeing AI tailored to specific industries, bringing about breakthroughs in healthcare, finance, transportation, and robotics. In healthcare, AI is improving diagnostic accuracy and personalizing treatment plans. In finance, it's enhancing fraud detection and automating trading. The transportation sector is witnessing smarter, safer autonomous vehicles, while robotics is seeing more efficient and adaptable machines. These industry-specific applications are not just improving efficiency but also revolutionizing how we approach problems.

![Tailored AI for Every Industry](./images/c78b2d72.png)

## Tackling Security and Ethical Challenges

With great power comes great responsibility, and AI is no exception. As AI systems become more integrated into our daily lives, security challenges are at the forefront. In 2025, there's a strong focus on developing robust systems to protect against cyber threats and ensure data privacy. Moreover, the regulatory landscape is evolving, emphasizing ethical AI, transparency, and accountability. These efforts are crucial in building trust and ensuring that AI technologies are used responsibly.

![Tackling Security and Ethical Challenges](./images/2d980b25.png)

## A Toolbox of AI Wonders

The proliferation of AI tools in 2025 is nothing short of amazing. From text and image processing to coding and video creation, there's an AI tool for almost every professional task. These tools are designed to make our lives easier, allowing us to focus on creativity and innovation. Whether you're a developer, a designer, or a content creator, AI is here to enhance your workflow and open up new possibilities.

In conclusion, 2025 is a landmark year for AI, marked by technological innovation and practical application. As we continue to explore the potential of AI, one thing is clear: the future is bright, and it's powered by artificial intelligence.